# Scores 2020

In [10]:
spark.stop()

# Setting paths

In [1]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when - is removed, remove also this line and adapt imports
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
if AMDOCS_SRC not in sys.path: 
    sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
    
#EXTERNAL_PATH = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel", "pykhaos", "external_lib")
EXTERNAL_PATH = "/var/SP/data/bdpmdses/churn/lib"
if EXTERNAL_PATH not in sys.path:
    sys.path.append(EXTERNAL_PATH)

import logging
logging.getLogger('matplotlib').setLevel(logging.WARNING)

    
import pykhaos.utils.notebooks as nb


RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    
    
    
#logger = my_project.logger

if not RUNNING_FROM_NOTEBOOK:
    args = my_project.arg_parser()

20200211-160429 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20200211_160429.log


In [2]:
import time
start_time = time.time()
app_name = "scores"

from churn_nrt.src.utils.spark_session import get_spark_session

sc, spark, sql_context = get_spark_session("scores")
sc.setLogLevel('WARN')

In [3]:
from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, \
        desc, asc, to_date, create_map, sum as sql_sum, substring, sort_array, split, month, dayofmonth
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType, DateType
from pyspark.sql.functions import array, regexp_extract
from itertools import chain
import argparse
import csv
import re
import subprocess
import sys
import time
from pyspark.sql import SparkSession, SQLContext

from pyspark.sql.functions import concat_ws, date_format, from_unixtime, \
    length, lit, lower, lpad, month, regexp_replace, translate, udf, unix_timestamp, year, when, upper
from pyspark.sql.utils import AnalysisException
from engine.general_functions import format_date, compute_diff_days, sum_horizontal
from collections import Counter
from pyspark.sql.types import StringType
from pykhaos.utils.date_functions import get_last_day_of_month, move_date_n_days, move_date_n_cycles, move_date_n_yearmonths
from churn.analysis.ccc_churn.engine.data_loader import get_port, get_ccc_data, get_tgs, get_all_ports
from churn.datapreparation.general.data_loader import get_active_services
from churn.analysis.ccc_churn.engine.reporter import compute_results, SAVING_PATH, init_writer, print_sheet
from churn.analysis.ccc_churn.app.run_ccc_churn_analysis import join_dfs

## Checks

In [14]:
 
df_onlymob = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_onlymob")
              .select("executed_at", "predict_closing_date").groupby("executed_at", "predict_closing_date").agg(sql_count("*").alias("onlymob")))
df_onlymob_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_onlymob").select("executed_at", "training_closing_date")
df_onlymob = df_onlymob.join(df_onlymob_params, on=["executed_at"], how="outer")
df_onlymob = df_onlymob.select("training_closing_date", "predict_closing_date", "onlymob")


df_mobandfbb = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_mobileandfbb")
                .select("executed_at", "predict_closing_date").groupby("executed_at", "predict_closing_date").agg(sql_count("*").alias("mobileandfbb")))

df_mobandfbb_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_mobileandfbb").select("executed_at", "training_closing_date")
df_mobandfbb = df_mobandfbb.join(df_mobandfbb_params, on=["executed_at"], how="outer")
df_mobandfbb = df_mobandfbb.select("training_closing_date", "predict_closing_date", "mobileandfbb")


df_others = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_others")
              .select("executed_at", "predict_closing_date").groupby("executed_at", "predict_closing_date").agg(sql_count("*").alias("others")))
df_others_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_others").select("executed_at", "training_closing_date")
df_others = df_others.join(df_others_params, on=["executed_at"], how="outer")
df_others = df_others.select("training_closing_date", "predict_closing_date", "others")


df_fbb = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=churn_preds_fbb")
            .select("executed_at", "predict_closing_date").groupby("executed_at", "predict_closing_date").agg(sql_count("*").alias("fbb")))
df_fbb_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=churn_preds_fbb").select("executed_at", "training_closing_date")
df_fbb = df_fbb.join(df_fbb_params, on=["executed_at"], how="outer")
df_fbb = df_fbb.select("training_closing_date", "predict_closing_date", "fbb")


df_deliv = (spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=delivery_churn")
            .withColumn("deliv_date", concat(col("year"), lpad(col("month"), 2, '0'), lpad(col("day"), 2, '0')))
            .select("executed_at", "predict_closing_date", "deliv_date").groupby("deliv_date", "executed_at", "predict_closing_date").agg(sql_count("*").alias("deliv")))
df_deliv_params = spark.read.load("/data/attributes/vf_es/model_outputs/model_parameters/model_name=delivery_churn").select("executed_at", "training_closing_date")
df_deliv = df_deliv.join(df_deliv_params, on=["executed_at"], how="outer")
df_deliv = df_deliv.select("training_closing_date", "predict_closing_date", "deliv", "deliv_date")



df_all = df_onlymob.join(df_mobandfbb, on=["predict_closing_date", "training_closing_date"], how="outer")
df_all = df_all.join(df_others, on=["predict_closing_date", "training_closing_date"], how="outer")
df_all = df_all.join(df_fbb, on=["predict_closing_date", "training_closing_date"], how="outer")
df_all = df_all.join(df_deliv, on=["predict_closing_date", "training_closing_date"], how="outer")


df_all.sort(desc("predict_closing_date"), desc("training_closing_date")).show()



+--------------------+---------------------+-------+------------+------+-------+-------+----------+
|predict_closing_date|training_closing_date|onlymob|mobileandfbb|others|    fbb|  deliv|deliv_date|
+--------------------+---------------------+-------+------------+------+-------+-------+----------+
|            20200207|   20191207to20191207|1303194|     3780223|671113|2222775|5754530|  20200214|
|            20200131|   20191130to20191130|1302108|     3782148|671112|2224387|5755368|  20200207|
|            20200121|   20191121to20191121|1305146|     3787517|671973|2227765|5764636|  20200131|
|            20200114|   20191114to20191114|1306227|     3792449|672478|2230165|5771154|  20200124|
|            20200107|   20191107to20191107|1307608|     3796679|673204|2232737|5777491|  20200117|
|            20191231|   20191031to20191031|1309533|     3799572|673658|2234718|5782763|  20200110|
|            20191221|   20191021to20191021|1309918|     3803404|674033|2237498|5787355|  20200103|


# BASE COMPARISON

In [19]:
from churn_nrt.src.data.customer_base import CustomerBase
from churn_nrt.src.data_utils.base_filters import get_mobile_base


df_bd = get_mobile_base(spark, date_="20190930", save=False)
#df_bd.where(col("NUM_CLIENTE")=="954345290").show()
df_bd.count()


# df_nrt_cust_base = spark.read.load("/data/udf/vf_es/churn_nrt/customer_base/year=2019/month=9/day=30")
# df_nrt_cust_base.count()


[DataTemplate] __init__ | module name = customer_base | path = /data/udf/vf_es/churn_nrt/customer_base | verbose = True
[DataTemplate] get_module | module customer_base - for closing_day=20190930 save=False save_others=False force_gen=False
[DataTemplate] get_module | args: () | kwargs: {}
[CustomerBase] is_default_module | args: () | kwargs: {}
[DataTemplate] get_module | Not found a module - '/data/udf/vf_es/churn_nrt/customer_base/year=2019/month=9/day=30'. Starting generation...
[CustomerBase] Get get_customer_base_segment base segment | date 20190930 save_others False
[CustomerBase] __get_customer_base date_=20190930 save_others=False add_columns_customer=None
[DataTemplate] __init__ | module name = customer | path = /data/udf/vf_es/churn_nrt/customer | verbose = True
[DataTemplate] get_module | module customer - for closing_day=20190930 save=False save_others=False force_gen=False
[DataTemplate] get_module | args: () | kwargs: {'add_columns': None}
[Customer] is_default_module | 

5871315

In [6]:
df_bd_anon = spark.read.load("/data/attributes/vf_es/model_outputs/model_scores/model_name=mobile_base/year=2019/month=9/day=30") #"/user/csanc109/data/mobile_base_20190930_parquet/")
df_bd_anon.count()

5842368

In [ ]:
df_bd = spark.read.option("delimeter", "|").option("header",False).csv("/user/csanc109/data/mobile_base_20190930_desanonim/Mobile_Base-20190930.csv")
headers = ['executed_at',
 'model_executed_at',
 'predict_closing_date',
        'msisdn',   
        'client_id'  , 
           'nif',
 'model_output',
 'scoring',
 'prediction',
 'extra_info',
 'time',
 
 
 ]
df_bd=df_bd.toDF(*headers)
# df_bd.count() # 5842368
from pyspark.sql.functions import substring_index, posexplode, split
EXTRA_INFO_COLS = ['srv_basic',
 'rgu',
 'CLASE_CLI_COD_CLASE_CLIENTE',
 'COD_ESTADO_GENERAL',
 'TARIFF']

for ii, col_ in enumerate(EXTRA_INFO_COLS):
    df_bd = df_bd.withColumn(col_, split("extra_info", ";")[ii])



In [ ]:
df_insi = spark.read.option("delimiter", "|").option("header",True).csv("/user/csanc109/data/mobile_base_20190930_insigths/mobile_base_20190930_insights.csv")
# for col_ in df_insi.columns:
#     if col_ == "MSISDN":
#         df_insi = df_insi.withColumnRenamed(col_, col_.lower())
#     else:
#         df_insi = df_insi.withColumnRenamed(col_, col_.lower()+"_insi")
        
# df_insi.count() # 5583563
# df_insi = df_insi.withColumn("campo1", col("msisdn"))
# df_insi = df_insi.withColumn("campo2", col("num_cliente"))
# df_insi = df_insi.withColumn("campo3", col("nif_cliente"))
# df_insi.columns

In [20]:
df_insi_anon = spark.read.load("/data/raw/vf_es/lookup/MOBILE_BASE/1.0/parquet/year=2020/month=1/day=17")

df_insi_anon.count()

5502418

### En BD y no en Insi

In [21]:
df_labels_cross = df_insi_anon.join(df_bd, ['msisdn'], 'right').where(df_insi_anon['msisdn'].isNull())
df_labels_cross = df_labels_cross.cache()
#df_labels_cross.select("msisdn").show()

print("volumen en BD y no en Insi", df_labels_cross.count())

('volumen en BD y no en Insi', 561072)


### En Insi y no en BD

In [22]:
df_labels_cross2 = df_bd.select("msisdn").join(df_insi_anon, ['msisdn'], 'right').where(df_bd['msisdn'].isNull())
df_labels_cross2 = df_labels_cross2.cache()
#df_labels_cross2.select("msisdn", "nif_cliente", "num_cliente").show()
print("volumen en Insi y no en BD", df_labels_cross2.count())

('volumen en Insi y no en BD', 192197)


### Volumen comun

In [23]:
df_labels_cross3 = df_insi_anon.join(df_bd, ['msisdn'], 'inner')
df_labels_cross3 = df_labels_cross3.cache()
print("volumen comun", df_labels_cross3.count())

('volumen comun', 5310242)


In [ ]:
df_labels_cross3.select("msisdn", "nif_cliente", "num_cliente").show()


In [ ]:
df_labels_cross3.select("msisdn", "clase_cli_cod_clase_cliente").show()

In [ ]:
df_labels_cross.groupby("CLASE_CLI_COD_CLASE_CLIENTE", 'COD_ESTADO_GENERAL').agg(sql_count("*").alias("count")).sort(desc("count")).show()

In [ ]:
df_labels_cross2.groupby('serv_basico_insi','tarifa_insi',).agg(sql_count("*").alias("count")).sort(desc("count")).show()

# navcomp exploration

In [ ]:
df_pasoprevio = spark.read.load("/data/udf/vf_es/netscout/dailyMSISDNApplicationName/year=2019/month=4/day=7")
df_pasoprevio.columns
df_pasoprevio2 = spark.read.load("/data/udf/vf_es/netscout/dailyMSISDNApplicationName/year=2019/month=10/day=7")
df_pasoprevio2.columns

set(df_pasoprevio.columns) ^ set(df_pasoprevio2.columns)

In [ ]:
df_data_limit200 = spark.read.load("/data/attributes/vf_es/return_feed/data_navigation/year=2019/month=10/day=7").limit(200)
msisdns = list(set(df_data_limit200.select("subscriber_msisdn").rdd.map(lambda x: x['subscriber_msisdn']).collect()))
#msisdns

In [ ]:
apps_names = list(set(spark.read.load("/data/attributes/vf_es/return_feed/data_navigation/year=2019/month=10/day=7").select("application_name").rdd.map(lambda x: x['application_name']).collect()))
apps_names



In [ ]:
(spark.read.load("/data/udf/vf_es/netscout/dailyMSISDNApplicationName/year=2019/month=10/day=7")
 .where(col("subscriber_msisdn").isin(["34683771231", "34695709335"]))
 .where( (col("SUM_userplane_upload_bytes_count") + col("SUM_userplane_download_bytes_count"))> 524288)
 .where(col("application_name").isin(apps_names)).sort(desc("subscriber_msisdn")).show())


In [ ]:
(spark.read.load("/data/attributes/vf_es/return_feed/data_navigation/year=2019/month=10/day=7")
 .where(col("subscriber_msisdn").isin(["34683771231", "34695709335"]))
 .where(col("application_name").isin(apps_names)).sort(desc("subscriber_msisdn")).show())

In [ ]:
spark.read.load("/data/attributes/vf_es/return_feed/data_navigation/year=2019/month=10/day=7").where(col("subscriber_msisdn").isin(["34683771231", "34695709335"])).show()



In [ ]:
operators = ["JAZZTEL", "LOWI", "MASMOVIL", "MOVISTAR", "O2", "ORANGE", "PEPEPHONE", "VODAFONE", "YOIGO"]
apps_names = ["WEB_"+op+"_"+p for op in operators for p in ["HTTP", "HTTPS"]]
apps_names


# apps_names = [u'WEB_O2_HTTPS',
#  u'WEB_MOVISTAR_HTTPS',
#  u'WEB_VODAFONE_HTTPS',
#  u'WEB_YOIGO_HTTP',
#  u'WEB_LOWI_HTTPS',
#  u'WEB_ORANGE_HTTP',
#  u'WEB_JAZZTEL_HTTP',
#  u'WEB_MASMOVIL_HTTP',
#  u'WEB_VODAFONE_HTTP',
#  u'WEB_MOVISTAR_HTTP',
#  u'WEB_ORANGE_HTTPS',
#  u'WEB_PEPEPHONE_HTTPS',
#  u'WEB_PEPEPHONE_HTTP',
#  u'WEB_JAZZTEL_HTTPS',
#  u'WEB_LOWI_HTTP',
#  u'WEB_MASMOVIL_HTTPS',
#  u'WEB_YOIGO_HTTPS',
#  u'WEB_O2_HTTP']

In [ ]:
(spark.read.load("/data/udf/vf_es/netscout/dailyMSISDNApplicationName/year=2019/month=10/day=17").where(col("application_name").isin(apps_names)).where(col("subscriber_msisdn").isNotNull())
           .withColumn("data",col("SUM_userplane_upload_bytes_count")+col("SUM_userplane_download_bytes_count"))
           .where(col("data")>lit(524288))
           .count())

In [ ]:
(spark.read.load("/data/udf/vf_es/netscout/dailyMSISDNApplicationName/year=2019/month=10/day=17").where(col("application_name").isin(apps_names))
           .withColumn("data",col("SUM_userplane_upload_bytes_count")+col("SUM_userplane_download_bytes_count"))
           .where(col("data")>lit(524288)).where(col("subscriber_msisdn").isNull()).count())

In [ ]:
(spark.read.load("/data/attributes/vf_es/return_feed/data_navigation/year=2019/month=10/day=17")
           .where(col("application_name").isin(apps_names)).count())

In [ ]:
len(msisdns)

In [ ]:
spark.read.load("/data/udf/vf_es/netscout/dailyMSISDNApplicationName/year=2019/month=10/day=7").where( (col("application_name").isin(apps_names)) & (col("subscriber_msisdn").isNotNull())).withColumn("data",col("SUM_userplane_upload_bytes_count")+col("SUM_userplane_download_bytes_count")).where(col("data")>lit(524288)).agg(sql_sum("data")).show(100)

In [ ]:
spark.read.load("/data/attributes/vf_es/return_feed/data_navigation/year=2019/month=10/day=7").withColumn("data",col("SUM_userplane_upload_bytes_count")+col("SUM_userplane_download_bytes_count")).agg(sql_sum("data")).show(100)


In [ ]:
from churn.analysis.triggers.navcomp.navcomp_utils import get_navcomp_attributes

starting_date = "20191001"
process_date = "20191015"

df_orig = get_navcomp_attributes(spark, starting_date, process_date, level="msisdn", suffix="", orig_path=True)
df_new = get_navcomp_attributes(spark, starting_date, process_date, level="msisdn", suffix="", orig_path=False)


msisdns = [u'34602579459',
 u'34699265865',
 u'34637118046',
 u'34643541635',
 u'34667016219',
 u'34603522398',
 u'34612028463',
 u'34602172915',
 u'34638122635',
 u'34612661085',
 u'34621389630',
 u'34626735798',
 u'34683771231',
 u'34664804027',
 u'34612610069',
 u'34639483976',
 u'34652601172',
 u'34609921178',
 u'34661956702',
 u'882393230430821',
 u'34639513800',
 u'34603357360',
 u'34652976638',
 u'34653975428',
 u'34612139344',
 u'34657565248',
 u'34656504103',
 u'34615956180',
 u'34608183564',
 u'34658555004',
 u'34650548809',
 u'34603997436',
 u'34694917249',
 u'34655457739',
 u'34679067766',
 u'34655551506',
 u'4534296369',
 u'34688510057',
 u'34692843710',
 u'34615977864',
 u'34694252365',
 u'34750363217',
 u'34621848425',
 u'34647631129',
 u'34661980383',
 u'34698409737',
 u'34655950914',
 u'34670819047',
 u'34602342056',
 u'34672405851',
 u'34697093354',
 u'34676843005',
 u'34607755410',
 u'34643746223',
 u'34624075045',
 u'34664164957',
 u'34640055967',
 u'34699436991',
 u'34615040780',
 u'34649505208',
 u'34630228529',
 u'34641787568',
 u'34695261294',
 u'34650852676',
 u'34651786844',
 u'34615406433',
 u'34661414103',
 u'34635496504',
 u'34650886571',
 u'34684878681',
 u'34679830577',
 u'34624664265',
 u'34686333247',
 u'34619574709',
 u'34661626345',
 u'34698930485',
 u'34643290334',
 u'34607224467',
 u'34611273565',
 u'34659755182',
 u'34626409295',
 u'34625087446',
 u'34697793869',
 u'34620566839',
 u'34647541941',
 u'34605096525',
 u'34650504179',
 u'34625289088',
 u'34683210998',
 u'34656319159',
 u'34630720603',
 u'447741382752',
 u'34683152899',
 u'34602929199',
 u'34692822948',
 u'34622813172',
 u'34670572823',
 u'34789958445',
 u'34688965869',
 u'34680347447',
 u'34664508482',
 u'491789031376',
 u'34609848759',
 u'34627904079',
 u'34655266152',
 u'34610455798',
 u'34695261579',
 u'34682884563',
 u'34666763747',
 u'34604753014',
 u'34682473201',
 u'34631727443',
 u'34610441363',
 u'882396873366857',
 u'34636544644',
 u'34679518709',
 u'34649438836',
 u'34654761792',
 u'34640763250',
 u'34693958858',
 u'34634959019',
 u'34642901478',
 u'34606813261',
 u'34635773177',
 u'34667060855',
 u'34628330469',
 u'34627906966',
 u'34609348547',
 u'34699593370',
 u'34601186557',
 u'34684669501',
 u'34693123558',
 u'34653037531',
 u'34679250802',
 u'34688200353',
 u'34611425181',
 u'34696064244',
 u'34676455684',
 u'34693045687',
 u'34683627907',
 u'34645462158',
 u'34601143234',
 u'34648188487',
 u'34629999175',
 u'34625299080',
 u'34693494584',
 u'491795499178',
 u'34692124140',
 u'34683818952',
 u'34600503065',
 u'34611822026',
 u'34613046218',
 u'34699581645',
 u'34664571843',
 u'34662072310',
 u'34656348015',
 u'34616934022',
 u'34636099471',
 u'34666271584',
 u'34670942078',
 u'34670472204',
 u'34663565093',
 u'34667039830',
 u'34688385443',
 u'34640854236',
 u'34609934152',
 u'34628964902',
 u'34620534832',
 u'34619282211',
 u'34640265051',
 u'34632300147',
 u'34691150155',
 u'34681535231',
 u'34693190094',
 u'34644285982',
 u'34620270626',
 u'34654243256',
 u'34662901998',
 u'34600732856',
 u'34656334691',
 u'34658894953',
 u'34610703900',
 u'34601426823',
 u'34601761332',
 u'34675069520',
 u'34654207810',
 u'34694845333',
 u'34613742893',
 u'34600038285',
 u'34606950109',
 u'34624679794',
 u'34637879188',
 u'34689218659',
 u'34643243869',
 u'34601168324',
 u'34669574220',
 u'34648207731',
 u'34613234875',
 u'34683590410',
 u'34634399909']


msisdns = [re.sub(r'^34', '', mm) for mm in msisdns]

df_orig = df_orig.where(col("msisdn").isin(msisdns))
df_new = df_new.where(col("msisdn").isin(msisdns))

df_orig = df_orig.show(200, truncate=False)
df_new = df_new.show(200, truncate=False)



In [ ]:
data_service_param.count()

In [ ]:
print(df_table.count(), df_table.distinct().count())

In [ ]:
print(data_service_param.count(), data_service_param.distinct().count())

In [ ]:
LOC_RT_PATH = '/data/udf/vf_es/ref_tables/amdocs_ids/'
LOC_RT_PARAM_OW_SERVICES= LOC_RT_PATH + 'PARAM_OW_SERVICES.TXT'
data_service_param = spark.read.csv(LOC_RT_PARAM_OW_SERVICES, header=True,sep='\t')
data_service_param.where(col("COD_SERVICIO").isin("TRDNP", "VFETC", "TESTH", "DTVC5")).show(truncate=False)

In [ ]:
from pyspark.sql.functions import  (sum as sql_sum, countDistinct, trim
                                    ,max, greatest , split,desc, col, current_date
                                    , datediff, lit, translate, udf
                                    , when, concat_ws, concat, decode, length
                                    , substring, to_date, regexp_replace, lpad
                                    , hour, date_format, row_number
                                    , expr, coalesce, udf, year)
from pyspark.sql.window import Window
import datetime
from engine.constants import UNKNOWN_TAG, OTHERS_TAG, L2_SUFFIX, DESC_TARIFF
from engine.general_functions import norm_values, format_date, compute_diff_days, sum_horizontal
from engine.amdocs_ids_config import path_customer
def is_null_date(fecha):
    return year(fecha) == 1753

In [ ]:
LOC_RT_PATH = '/data/udf/vf_es/ref_tables/amdocs_ids/'
LOC_RT_PARAM_OW_SERVICES = LOC_RT_PATH + 'PARAM_OW_SERVICES.TXT'
data_service_param =  spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true',delimiter='\t').load(LOC_RT_PARAM_OW_SERVICES).select("RGU", "COD_SERVICIO", "TIPO").distinct()
df_table = spark.read.load("/data/raw/vf_es/customerprofilecar/WEBSERVICES/1.0/parquet").select("COD_SERVICIO", "RGU", "AMBITO").withColumnRenamed("RGU", "RGU_table").distinct()
df_table = df_table.withColumn("RGU_table", when(col("RGU_table")=="F - Internet (BA)", "fbb").when(col("RGU_table")=="F - Televisión (TV)", "tv").when(col("RGU_table")=="M - Móvil (MV)", "mobile").otherwise(col("RGU_table")))

df_all = df_table.join(data_service_param, on=["COD_SERVICIO"], how="outer")
df_all = df_all.where( ~((col("RGU_table")=="mobile") & (col("RGU")=="mobile")))
df_all = df_all.where(col("RGU") == "mobile")

df_all.show(2000)

In [ ]:
A = ['nif_cliente',
 'msisdn',
 'rgu',
 'NUM_CLIENTE',
 'cod_estado_general',
 'srv_basic',
 'clase_cli_cod_clase_cliente',
 'segment_nif',
 'nb_fbb_services_nif',
 'nb_mobile_services_nif',
 'nb_tv_services_nif',
 'nb_prepaid_services_nif',
 'nb_bam_mobile_services_nif',
 'nb_fixed_services_nif',
 'nb_bam_services_nif',
 'nb_rgus_nif',
 'rgus_list',
 'tgs_days_until_fecha_fin_dto',
 'tgs_has_discount']

In [23]:
metadata_sources =  ['navcomp', 'customer', 'ccc', 'spinners', "scores"]
from churn_nrt.src.projects.models.navcomp.metadata import METADATA_STANDARD_MODULES, get_metadata


from churn_nrt.src.data_utils.Metadata import Metadata
metadata_obj = Metadata(spark, get_metadata, ["msisdn"], metadata_sources)

#navcomp_tr_df = self.METADATA.fillna(navcomp_tr_df, sources=['scores'])

cols = None
sources = ["scores"]

if not cols and not sources:
    print("Calling fillna with sources=None and cols=Non. Filling nans for all sources ({})".format(",".join(self.METADATA_SOURCES)))
    sources = metadata_sources

if sources:
    df_metadata = metadata_obj.get_metadata_df(sources=sources)
    cols_sources = df_metadata.rdd.map(lambda x: x['feature']).collect()
else:
    cols_sources = []

cols = cols_sources if not cols else list(set(cols) | set(cols_sources))

cols

#apply_metadata(df_metadata, df, cols)




[DataTemplate] module name = scores | path = /data/udf/vf_es/churn_nrt/scores | verbose = True
[Metadata] Adding Scores metadata


[u'latest_score']